In [1]:
from pyvisa import ResourceManager
from piec.drivers.keysight81150a import Keysight81150a
from piec.drivers.keysightdsox3024a import Dsox3024a
rm = ResourceManager()

In [2]:
rm.list_resources()

('GPIB0::7::INSTR', 'GPIB0::8::INSTR')

In [3]:
awg = Keysight81150a("GPIB0::8::INSTR")
scope = Dsox3024a("GPIB0::7::INSTR")

In [4]:
awg.idn()

'Agilent Technologies,81150A,MY53821602,3.0.0.0-4.6\n'

In [5]:
scope.idn()

'AGILENT TECHNOLOGIES,DSO-X 3024A,MY63080078,02.65.2021030741\n'

In [6]:
#Setup functions
#Set up the pulse params
awg.configure_wf(channel='1', func="PULS", voltage="1.0", offset="0.0", frequency="1.0e7", duty_cycle="50")
#Sets the trigger source to manual
awg.configure_trigger(trigger_source="MAN")
#turns the output on
awg.output_enable(channel='1')

In [18]:
#Configure the scope
channel = 1

scope.initialize()
scope.configure_timebase(time_base_type='MAIN', reference='CENTer', time_scale=f'{100e-9}', position=f'{0}') #this should be made general
scope.configure_channel(channel=f'{channel}', voltage_scale=f'{1}', impedance='FIFT')#set both to 50ohm
scope.configure_trigger_characteristics(trigger_source='CHAN1', low_voltage_level='0.75', high_voltage_level='0.95', sweep='NORM')
scope.configure_trigger_edge(trigger_source='CHAN1', input_coupling='DC')

In [19]:
scope.initiate()

In [20]:
awg.send_software_trigger()


In [21]:
scope.setup_wf(points_mode="MAX")

In [22]:
bet = scope.query_wf()
#bet = scope.query_wf()

In [23]:
bet

({'format': np.int16(0),
  'type': np.int16(0),
  'points': np.int32(1000),
  'count': np.int32(1),
  'x_increment': np.float64(1e-09),
  'x_origin': np.float64(-5e-07),
  'x_reference': np.int32(0),
  'y_increment': np.float32(0.040201),
  'y_origin': np.float32(0.0),
  'y_reference': np.int32(0)},
 [np.float64(-5e-07),
  np.float64(-4.99e-07),
  np.float64(-4.979999999999999e-07),
  np.float64(-4.97e-07),
  np.float64(-4.96e-07),
  np.float64(-4.95e-07),
  np.float64(-4.94e-07),
  np.float64(-4.93e-07),
  np.float64(-4.92e-07),
  np.float64(-4.909999999999999e-07),
  np.float64(-4.9e-07),
  np.float64(-4.89e-07),
  np.float64(-4.88e-07),
  np.float64(-4.87e-07),
  np.float64(-4.86e-07),
  np.float64(-4.85e-07),
  np.float64(-4.839999999999999e-07),
  np.float64(-4.83e-07),
  np.float64(-4.82e-07),
  np.float64(-4.809999999999999e-07),
  np.float64(-4.8e-07),
  np.float64(-4.79e-07),
  np.float64(-4.78e-07),
  np.float64(-4.769999999999999e-07),
  np.float64(-4.76e-07),
  np.float64(-

In [15]:
bet = scope.query_wf()

In [ ]:
bet

In [14]:
#send the trigger
awg.send_software_trigger()